# Project 1: Milestone 2
### DSC680-T301 Applied Data Science
### Joshua Greenert
### 3/18/2023

## Data Preparation

In [17]:
# Set some required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pull in the data.
df_housing = pd.read_csv('Housing.csv')
df_hpi = pd.read_csv('HPI_master.csv')
df_state_stats = pd.read_excel('state_statistics_for_download.xls')

# Show the head.
df_state_stats.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Estimated Mean and Median House Prices for U.S...,NaN,NaN,NaN
1,Note: Underlying methodology and data sources ...,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,State,Year-Quarter,Average Price,Median Price
4,US,2000Q1,151780,125839


In [18]:
# Fix the state stats dataframe.
# Correct the row headers for the state stats.
df_state_stats.columns = df_state_stats.iloc[3]

#remove first row from DataFrame
df_state_stats = df_state_stats[4:]

# Remove all values of US from the State column
df_state_stats = df_state_stats[df_state_stats['State'] != 'US']

# Remove the rows with Q2, Q3, and Q4 data.
df_state_stats = df_state_stats[~df_state_stats['Year-Quarter'].str.contains('Q[234]')]

In [20]:
# Loop through the dataframe and replace each year-quarter with an actual date object.
for index, row in df_state_stats.iterrows():
    print(row['Year-Quarter'])


2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1
2000Q1
2001Q1
2002Q1
2003Q1
2004Q1
2005Q1
2006Q1
2007Q1
2008Q1
2009Q1
2010Q1